In [3]:
#importing libraries
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import time
import tensorflow as tf

%matplotlib inline

In [4]:
#utility function to reset graph
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Creating & Running a Graph

Define a composite function $f(w) = exp(w_{20} + w_{21}.exp(w_{10} + w_{11}.exp(w_{00} + w_{01}.x)))$

In [5]:
#implementing above function in tensorflow
def eq(w, x):
    f0 = tf.exp(w[0,0] + w[0,1]*w)
    f1 = tf.exp(w[1,0] + w[1,1]*f0)
    f2 = tf.exp(w[2,0] + w[2,1]*f1)
    
    return f2, f1, f0

In [27]:
#fancier implementation for the function with name scope
def eq(w, x):
    with tf.name_scope('f0_level') as scope0:
        f0 = tf.exp(w[0,0] + w[0,1]*w)
    with tf.name_scope('f1_level') as scope1:
        f1 = tf.exp(w[1,0] + w[1,1]*f0)
    with tf.name_scope('f2_level') as scope2:
        f2 = tf.exp(w[2,0] + w[2,1]*f1)
        
    return f2, f1, f0

In [10]:
# w0 is the point at which we want to compute the derivative
w0 = np.vstack((np.zeros(3), np.ones(3))).T
w0

array([[0., 1.],
       [0., 1.],
       [0., 1.]])

# Computing gradients using TensorFlow

In [28]:
reset_graph()

#define and initialize variables
w = tf.Variable(w0, name='w', dtype=tf.float32)
x = tf.Variable(1.0, name='x', dtype=tf.float32, trainable=False)

#define nodes for the values of the function at layers
f2, f1, f0 = eq(w, x)

#define node for the gradients by tensorflow's autodiff
gradient_function = tf.gradients(f2, w)

gradient_function

[<tf.Tensor 'gradients/AddN:0' shape=(3, 2) dtype=float32>]

In [29]:
#a node for the initializer
init = tf.global_variables_initializer()

#run session
t = time.time()

with tf.Session() as sess:
    sess.run(init)
    gradients, function_values = sess.run([gradient_function, [f2, f1, f0]])
    
print('Computation time: %f3.2 sec' %(time.time()-t))
print('Funciton Values: ',function_values)
print('Gradients: ',gradients)

Computation time: 0.0405173.2 sec
Funciton Values:  [array([[1.5154261e+01, 3.8142730e+06],
       [1.5154261e+01, 3.8142730e+06],
       [1.5154261e+01, 3.8142730e+06]], dtype=float32), array([[ 2.7182817, 15.154261 ],
       [ 2.7182817, 15.154261 ],
       [ 2.7182817, 15.154261 ]], dtype=float32), array([[1.       , 2.7182817],
       [1.       , 2.7182817],
       [1.       , 2.7182817]], dtype=float32)]
Gradients:  [array([[4.7137046e+08, 6.2849376e+08],
       [1.7340763e+08, 6.2849389e+08],
       [1.1442905e+07, 3.3053101e+08]], dtype=float32)]


# Visualizing above graph

In [25]:
#function for visualizing graph with Jupyter

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add()
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize tensorflow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
            <script>
                function load()
                {{document.getElementById("{id}").pbtxt = {data};}}
            </script>
            <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
            <div style="height:600px">
                <tf-graph-basic id="{id}"></tf-graph-basic>
            </div>
            """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
    
    iframe = """<iframe seamless style="width:1200px; height:620px; border:0" srcdoc="{}"></iframe>""".format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [31]:
show_graph(tf.get_default_graph())